In [ ]:
# to use virtual env
# python3 -m venv venv
# source venv/bin/activate

!pip3 install --upgrade pip
!pip3 install ultralytics opencv-python
!pip3 install pandas


In [ ]:
# download the dataset: https://www.kaggle.com/datasets/andrewmvd/car-plate-detection?resource=download
# rename the folder: "dataset"

In [14]:
import os
import shutil
import random
import time
import pandas as pd
from ultralytics import YOLO
import xml.etree.ElementTree as ET
import torch


In [ ]:
# process the annotations in YOLO text format
# split the data into train 70%, val 20% and test 10%

random.seed(42)

image_dir = 'dataset/images'
label_dir = 'dataset/annotations'

out_base = 'dataset_split'

def convert_xml_to_yolo(xml_path, txt_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    size = root.find('size')
    width = float(size.find('width').text)
    height = float(size.find('height').text)

    lines = []
    for obj in root.findall('object'):
        class_id = 0 

        bndbox = obj.find('bndbox')
        xmin = float(bndbox.find('xmin').text)
        ymin = float(bndbox.find('ymin').text)
        xmax = float(bndbox.find('xmax').text)
        ymax = float(bndbox.find('ymax').text)

        x_center = ((xmin + xmax) / 2) / width
        y_center = ((ymin + ymax) / 2) / height
        w = (xmax - xmin) / width
        h = (ymax - ymin) / height

        lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}")

    with open(txt_path, 'w') as f:
        f.write("\n".join(lines))

image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]
base_names = [os.path.splitext(f)[0] for f in image_files]

random.shuffle(base_names)
total = len(base_names)
train_end = int(0.7 * total)
val_end = int(0.9 * total)

splits = {
    'train': base_names[:train_end],
    'val': base_names[train_end:val_end],
    'test': base_names[val_end:]
}

for split in ['train', 'val', 'test']:
    os.makedirs(f'{out_base}/images/{split}', exist_ok=True)
    os.makedirs(f'{out_base}/labels/{split}', exist_ok=True)

for split, names in splits.items():
    for name in names:
        img_src = os.path.join(image_dir, f'{name}.png')
        lbl_src = os.path.join(label_dir, f'{name}.xml')

        img_dst = os.path.join(out_base, 'images', split, f'{name}.png')
        txt_dst = os.path.join(out_base, 'labels', split, f'{name}.txt')

        if os.path.exists(img_src):
            shutil.copy(img_src, img_dst)

        if os.path.exists(lbl_src):
            convert_xml_to_yolo(lbl_src, txt_dst)

print("Files split, grouped, and labels converted to YOLO TXT format")


Files split, grouped, and labels converted to YOLO TXT format


In [ ]:
if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"Using device: {device}")


Using device: mps


In [20]:
models = ["yolov8n.pt", "yolov8s.pt", "yolov8m.pt"]
data_yaml = "dataset_split/data.yaml"
epochs = 5

results = []

for model_name in models:
    print(f"\nTraining {model_name}...")
    model = YOLO(model_name)
    
    start_time = time.time()
    metrics = model.train(data=data_yaml, epochs=epochs)
    end_time = time.time()
    train_time = round(end_time - start_time, 2)
    
    map50 = metrics.results_dict.get("metrics/mAP50(B)", None)
    recall = metrics.results_dict.get("metrics/recall(B)", None)
    precision = metrics.results_dict.get("metrics/precision(B)", None)
    f1 = metrics.results_dict.get("metrics/f1(B)", None)
    
    results.append({
        "Model": model_name,
        "Precision": round(precision, 4) if precision else None,
        "Recall": round(recall, 4) if recall else None,
        "F1 Score": round(f1, 4) if f1 else None,
        "mAP50": round(map50, 4) if map50 else None,
        "Training Time (s)": train_time
    })

df = pd.DataFrame(results)
df.to_csv("yolo_model_comparison.csv", index=False)
display(df)



Training yolov8n.pt...
Ultralytics 8.3.174 🚀 Python-3.12.8 torch-2.7.1 CPU (Apple M1)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset_split/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train8, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=T

train: Scanning /Users/neslyn/Desktop/dl_project/dataset_split/labels/train.cache... 303 images, 0 backgrounds, 0 corrupt: 100%|██████████| 303/303 [00:00<?, ?it/s]

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 794.7±305.8 MB/s, size: 559.0 KB)



val: Scanning /Users/neslyn/Desktop/dl_project/dataset_split/labels/val.cache... 86 images, 0 backgrounds, 0 corrupt: 100%|██████████| 86/86 [00:00<?, ?it/s]

Plotting labels to /Users/neslyn/Desktop/dl_project/runs/detect/train8/labels.jpg... 


...

                   all         86         92      0.693      0.467       0.47      0.236



5 epochs completed in 1.365 hours.
Optimizer stripped from /Users/neslyn/Desktop/dl_project/runs/detect/train10/weights/last.pt, 52.0MB
Optimizer stripped from /Users/neslyn/Desktop/dl_project/runs/detect/train10/weights/best.pt, 52.0MB

Validating /Users/neslyn/Desktop/dl_project/runs/detect/train10/weights/best.pt...
Ultralytics 8.3.174 🚀 Python-3.12.8 torch-2.7.1 CPU (Apple M1)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [01:11<00:00, 23.99s/it]


                   all         86         92        0.8      0.728      0.739      0.384
Speed: 0.9ms preprocess, 819.6ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to /Users/neslyn/Desktop/dl_project/runs/detect/train10


,Model,Precision,Recall,F1 Score,mAP50,Training Time (s)
0,yolov8n.pt,0.8997,0.5853,None,0.7642,941.91
1,yolov8s.pt,0.9301,0.7235,None,0.8386,2354.00
2,yolov8m.pt,0.8001,0.7283,None,0.7388,4992.39


In [21]:
# train and validate model
# from the table above I could see yolo v8 small performs better

model = YOLO("yolov8s.pt")
model.train(data="dataset_split/data.yaml", epochs=7)
metrics = model.val(data="dataset_split/data.yaml")
print(metrics)


Ultralytics 8.3.174 🚀 Python-3.12.8 torch-2.7.1 CPU (Apple M1)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset_split/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=7, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train11, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, pro

train: Scanning /Users/neslyn/Desktop/dl_project/dataset_split/labels/train.cache... 303 images, 0 backgrounds, 0 corrupt: 100%|██████████| 303/303 [00:00<?, ?it/s]

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 558.9±95.3 MB/s, size: 559.0 KB)



val: Scanning /Users/neslyn/Desktop/dl_project/dataset_split/labels/val.cache... 86 images, 0 backgrounds, 0 corrupt: 100%|██████████| 86/86 [00:00<?, ?it/s]

Plotting labels to /Users/neslyn/Desktop/dl_project/runs/detect/train11/labels.jpg... 


...


7 epochs completed in 0.776 hours.
Optimizer stripped from /Users/neslyn/Desktop/dl_project/runs/detect/train11/weights/last.pt, 22.5MB
Optimizer stripped from /Users/neslyn/Desktop/dl_project/runs/detect/train11/weights/best.pt, 22.5MB

Validating /Users/neslyn/Desktop/dl_project/runs/detect/train11/weights/best.pt...
Ultralytics 8.3.174 🚀 Python-3.12.8 torch-2.7.1 CPU (Apple M1)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:31<00:00, 10.60s/it]


                   all         86         92      0.844       0.88      0.899      0.463
Speed: 0.7ms preprocess, 356.6ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to /Users/neslyn/Desktop/dl_project/runs/detect/train11
Ultralytics 8.3.174 🚀 Python-3.12.8 torch-2.7.1 CPU (Apple M1)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2412.8±1810.6 MB/s, size: 562.9 KB)


val: Scanning /Users/neslyn/Desktop/dl_project/dataset_split/labels/val.cache... 86 images, 0 backgrounds, 0 corrupt: 100%|██████████| 86/86 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:30<00:00,  5.09s/it]


                   all         86         92      0.855       0.87      0.899      0.465
Speed: 0.6ms preprocess, 340.2ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to /Users/neslyn/Desktop/dl_project/runs/detect/train112
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x103ca0b30>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003, 

In [ ]:
results = model.predict(source="dataset_split/images/test", conf=0.4, save=True)


2025-08-04 16:49:46.825 Python[62885:2526145] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


ConnectionError: 1/1: 0... Failed to read images from 0